<a href="https://colab.research.google.com/github/Silvio-0-1/ey-technical-training/blob/main/2026-01-19/Telecom_Network_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CASE STUDY 7**

### **Title:** Telecom Network Quality and Customer Experience Analytics using PySpark

In [5]:
# Initialize Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TelecomNetworkQuality") \
    .getOrCreate()

In [6]:
from pyspark.sql.functions import (
    col, trim, upper, lower, when, regexp_replace,
    avg, count, sum as _sum,
    to_timestamp, coalesce,
    row_number, lag,
    round, expr
)
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

## **PHASE 1: INGESTION**


In [7]:
# 1. Read network_logs.csv as all StringType
schema_string = StructType([
    StructField("event_id", StringType(), True),
    StructField("subscriber_id", StringType(), True),
    StructField("tower_id", StringType(), True),
    StructField("city", StringType(), True),
    StructField("network_type", StringType(), True),
    StructField("signal_strength", StringType(), True),
    StructField("download_speed_mbps", StringType(), True),
    StructField("upload_speed_mbps", StringType(), True),
    StructField("latency_ms", StringType(), True),
    StructField("call_drop", StringType(), True),
    StructField("event_time", StringType(), True),
    StructField("device_type", StringType(), True)
])

df_raw = spark.read.csv("/content/network_logs.csv", header=True, schema=schema_string)

In [8]:
# 2. Print schema and row count
df_raw.printSchema()
print(f"Raw Row Count: {df_raw.count()}")

root
 |-- event_id: string (nullable = true)
 |-- subscriber_id: string (nullable = true)
 |-- tower_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- network_type: string (nullable = true)
 |-- signal_strength: string (nullable = true)
 |-- download_speed_mbps: string (nullable = true)
 |-- upload_speed_mbps: string (nullable = true)
 |-- latency_ms: string (nullable = true)
 |-- call_drop: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- device_type: string (nullable = true)

Raw Row Count: 90670


In [9]:
# 3. Show sample rows
df_raw.show(5)

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
| E100000|        S5975|    T837|Bangalore|          3G|        invalid|               NULL|             NULL|   invalid|      YES|01/01/2026 00:00:00|     Android|
| E100001|        S3537|    T283|Hyderabad|          5G|            -83|             124.07|            41.26|       114|       NO|2026-01-01 00:00:03|FeaturePhone|
| E100002|        S1629|    T877|     Pune|          4G|            -72|              41.01|             3.36|       221|       NO|2026-01-01 00:00:06|FeaturePhone|
| E100003|

## **PHASE 2: CLEANING**

In [10]:
# 1. Trim string columns
df_trimmed = df_raw.select([trim(col(c)).alias(c) for c in df_raw.columns])

df_trimmed.show(5)

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
| E100000|        S5975|    T837|Bangalore|          3G|        invalid|               NULL|             NULL|   invalid|      YES|01/01/2026 00:00:00|     Android|
| E100001|        S3537|    T283|Hyderabad|          5G|            -83|             124.07|            41.26|       114|       NO|2026-01-01 00:00:03|FeaturePhone|
| E100002|        S1629|    T877|     Pune|          4G|            -72|              41.01|             3.36|       221|       NO|2026-01-01 00:00:06|FeaturePhone|
| E100003|

In [11]:
# 2. Normalize string fields (city, network_type, device_type, call_drop)
df_normalized = df_trimmed.withColumn("city", upper(col("city"))) \
       .withColumn("network_type", upper(col("network_type"))) \
       .withColumn("device_type", upper(col("device_type"))) \
       .withColumn("call_drop", upper(col("call_drop")))

df_normalized.show(5)

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
| E100000|        S5975|    T837|BANGALORE|          3G|        invalid|               NULL|             NULL|   invalid|      YES|01/01/2026 00:00:00|     ANDROID|
| E100001|        S3537|    T283|HYDERABAD|          5G|            -83|             124.07|            41.26|       114|       NO|2026-01-01 00:00:03|FEATUREPHONE|
| E100002|        S1629|    T877|     PUNE|          4G|            -72|              41.01|             3.36|       221|       NO|2026-01-01 00:00:06|FEATUREPHONE|
| E100003|

In [12]:
# 3. Clean numeric fields safely
df_cleaned = (
    df_normalized
    .withColumn(
        "signal_strength",
        when(
            regexp_replace(col("signal_strength"), "[^0-9-]", "") == "",
            None
        ).otherwise(
            regexp_replace(col("signal_strength"), "[^0-9-]", "").cast(IntegerType())
        )
    )
    .withColumn(
        "download_speed_mbps",
        when(
            regexp_replace(col("download_speed_mbps"), "[^0-9.]", "") == "",
            None
        ).otherwise(
            regexp_replace(col("download_speed_mbps"), "[^0-9.]", "").cast(DoubleType())
        )
    )
    .withColumn(
        "upload_speed_mbps",
        when(
            regexp_replace(col("upload_speed_mbps"), "[^0-9.]", "") == "",
            None
        ).otherwise(
            regexp_replace(col("upload_speed_mbps"), "[^0-9.]", "").cast(DoubleType())
        )
    )
    .withColumn(
        "latency_ms",
        when(
            regexp_replace(col("latency_ms"), "[^0-9]", "") == "",
            None
        ).otherwise(
            regexp_replace(col("latency_ms"), "[^0-9]", "").cast(IntegerType())
        )
    )
)

df_cleaned.show(5)

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+
| E100000|        S5975|    T837|BANGALORE|          3G|           NULL|               NULL|             NULL|      NULL|      YES|01/01/2026 00:00:00|     ANDROID|
| E100001|        S3537|    T283|HYDERABAD|          5G|            -83|             124.07|            41.26|       114|       NO|2026-01-01 00:00:03|FEATUREPHONE|
| E100002|        S1629|    T877|     PUNE|          4G|            -72|              41.01|             3.36|       221|       NO|2026-01-01 00:00:06|FEATUREPHONE|
| E100003|

In [13]:
# 4. Parse event_time with multiple formats
df = df_cleaned.withColumn(
    "event_time_clean",
    expr("""
        coalesce(
            try_to_timestamp(event_time, 'yyyy-MM-dd HH:mm:ss'),
            try_to_timestamp(event_time, 'dd/MM/yyyy HH:mm:ss'),
            try_to_timestamp(event_time, 'yyyy/MM/dd HH:mm:ss')
        )
    """)
)

df.show(5)

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|   event_time_clean|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+
| E100000|        S5975|    T837|BANGALORE|          3G|           NULL|               NULL|             NULL|      NULL|      YES|01/01/2026 00:00:00|     ANDROID|2026-01-01 00:00:00|
| E100001|        S3537|    T283|HYDERABAD|          5G|            -83|             124.07|            41.26|       114|       NO|2026-01-01 00:00:03|FEATUREPHONE|2026-01-01 00:00:03|
| E100002|        S1629|    T877|     PUNE|          4G|            -72|   

## **PHASE 3: VALIDATION**

In [28]:
# 1. Count invalid values for each numeric field
invalid_counts = df.select(
    _sum(col("signal_strength").isNull().cast("int")).alias("invalid_signal_strength"),
    _sum(col("download_speed_mbps").isNull().cast("int")).alias("invalid_download_speed"),
    _sum(col("upload_speed_mbps").isNull().cast("int")).alias("invalid_upload_speed"),
    _sum(col("latency_ms").isNull().cast("int")).alias("invalid_latency")
)
print("Invalid Value Counts:")
invalid_counts.show()

Invalid Value Counts:
+-----------------------+----------------------+--------------------+---------------+
|invalid_signal_strength|invalid_download_speed|invalid_upload_speed|invalid_latency|
+-----------------------+----------------------+--------------------+---------------+
|                   4773|                  6934|                2925|           2451|
+-----------------------+----------------------+--------------------+---------------+



In [15]:
# 2. Count invalid timestamps
print("Invalid timestamps:")
df.filter(col("event_time_clean").isNull()).count()

Invalid timestamps:


808

In [16]:
# 3. Remove duplicate logs
df = df.dropDuplicates(["event_id"])
print(f"Count after removal: {df.count()}")

Count after removal: 90670


## **PHASE 4: NETWORK KPIS**

In [17]:
# 1. Average download speed per city
avg_download_city = df.groupBy("city") \
    .agg(round(avg("download_speed_mbps"), 2).alias("avg_download_speed"))
avg_download_city.show()

+---------+------------------+
|     city|avg_download_speed|
+---------+------------------+
|  KOLKATA|             75.22|
|BANGALORE|              75.7|
|    DELHI|             76.21|
|HYDERABAD|             74.89|
|  CHENNAI|             76.01|
|     PUNE|             75.16|
|   MUMBAI|             75.88|
+---------+------------------+



In [18]:
# 2. Average latency per city
avg_latency_city = df.groupBy("city") \
    .agg(round(avg("latency_ms"), 2).alias("avg_latency"))

avg_latency_city.show()

+---------+-----------+
|     city|avg_latency|
+---------+-----------+
|  KOLKATA|     155.21|
|BANGALORE|     156.44|
|    DELHI|     154.46|
|HYDERABAD|     154.86|
|  CHENNAI|     155.56|
|     PUNE|     154.93|
|   MUMBAI|     153.58|
+---------+-----------+



In [19]:
# 3. Call drop rate per city
call_drop_city = df.groupBy("city") \
    .agg(
        round(
            _sum(when(col("call_drop") == "YES", 1).otherwise(0)) / count("*"),
            3
        ).alias("call_drop_rate")
    )

call_drop_city.show()

+---------+--------------+
|     city|call_drop_rate|
+---------+--------------+
|  KOLKATA|         0.049|
|BANGALORE|         0.049|
|    DELHI|         0.053|
|HYDERABAD|         0.052|
|  CHENNAI|         0.045|
|     PUNE|         0.049|
|   MUMBAI|         0.051|
+---------+--------------+



In [20]:
# 4. Call drop rate per tower
call_drop_tower = df.groupBy("tower_id") \
    .agg(
        round(
            _sum(when(col("call_drop") == "YES", 1).otherwise(0)) / count("*"),
            3
        ).alias("call_drop_rate")
    )

call_drop_tower.show(5)

+--------+--------------+
|tower_id|call_drop_rate|
+--------+--------------+
|    T929|         0.033|
|    T352|         0.037|
|    T947|         0.019|
|    T855|         0.062|
|    T590|         0.048|
+--------+--------------+
only showing top 5 rows


In [21]:
# 5. Top 10 worst towers
# Criteria: High Drop Rate, High Latency, Low Download Speed
worst_towers = df.groupBy("tower_id") \
    .agg(
        round(avg("latency_ms"), 2).alias("avg_latency"),
        round(avg("download_speed_mbps"), 2).alias("avg_download_speed"),
        round(
            _sum(when(col("call_drop") == "YES", 1).otherwise(0)) / count("*"),
            3
        ).alias("call_drop_rate")
    ) \
    .orderBy(col("call_drop_rate").desc(),
             col("avg_latency").desc(),
             col("avg_download_speed").asc()) \
    .limit(10)

worst_towers.show()

+--------+-----------+------------------+--------------+
|tower_id|avg_latency|avg_download_speed|call_drop_rate|
+--------+-----------+------------------+--------------+
|    T653|     157.03|             76.39|         0.129|
|    T275|     145.89|              66.6|         0.121|
|    T358|     165.48|             76.24|          0.12|
|    T538|     166.14|              70.5|         0.118|
|    T699|     160.81|             81.61|         0.118|
|    T697|     148.62|             76.86|         0.116|
|    T252|     161.64|             82.42|         0.111|
|    T533|     147.16|             74.07|         0.111|
|    T172|     156.95|             78.74|         0.107|
|    T119|     155.08|             77.85|         0.106|
+--------+-----------+------------------+--------------+



## **PHASE 5: CUSTOMER EXPERIENCE**

In [22]:
# Compute metrics for each subscriber_id
subscriber_metrics = df.groupBy("subscriber_id") \
    .agg(
        count("*").alias("event_count"),
        round(avg("download_speed_mbps"), 2).alias("avg_download_speed"),
        round(avg("latency_ms"), 2).alias("avg_latency"),
        _sum(when(col("call_drop") == "YES", 1).otherwise(0)).alias("call_drop_count")
    )

subscriber_metrics.show(5)

+-------------+-----------+------------------+-----------+---------------+
|subscriber_id|event_count|avg_download_speed|avg_latency|call_drop_count|
+-------------+-----------+------------------+-----------+---------------+
|        S2422|          9|             58.32|     157.56|              0|
|        S1828|         17|             69.37|     110.35|              0|
|        S8433|          9|             51.45|     201.38|              1|
|        S2414|          4|             70.46|     186.67|              0|
|        S7616|          9|             92.68|     184.11|              0|
+-------------+-----------+------------------+-----------+---------------+
only showing top 5 rows


In [23]:
# Poor experience logic
# Logic defined: High drops (>=3), Low speed (<5 Mbps), High latency (>200ms)
poor_experience_users = subscriber_metrics.filter(
    (col("call_drop_count") >= 3) |
    (col("avg_download_speed") < 5) |
    (col("avg_latency") > 200)
)

poor_experience_users.show(5)

+-------------+-----------+------------------+-----------+---------------+
|subscriber_id|event_count|avg_download_speed|avg_latency|call_drop_count|
+-------------+-----------+------------------+-----------+---------------+
|        S8433|          9|             51.45|     201.38|              1|
|        S7972|         12|             85.24|     144.58|              3|
|        S8711|          8|             74.95|     231.13|              0|
|        S8207|          4|             91.02|      213.5|              0|
|        S8378|         14|             91.76|     204.29|              4|
+-------------+-----------+------------------+-----------+---------------+
only showing top 5 rows


## **PHASE 6: WINDOW FUNCTIONS**

In [24]:
# 1. Rank towers within each city by call drop rate
tower_city_window = Window.partitionBy("city").orderBy(col("call_drop_rate").desc())

tower_ranked = call_drop_city.withColumn(
    "tower_rank",
    row_number().over(tower_city_window)
)
tower_ranked.show(5)

+---------+--------------+----------+
|     city|call_drop_rate|tower_rank|
+---------+--------------+----------+
|BANGALORE|         0.049|         1|
|  CHENNAI|         0.045|         1|
|    DELHI|         0.053|         1|
|HYDERABAD|         0.052|         1|
|  KOLKATA|         0.049|         1|
+---------+--------------+----------+
only showing top 5 rows


In [29]:
# 2. Rank subscribers within each city by worst experience
subscriber_city_df = df.join(subscriber_metrics, "subscriber_id")

subscriber_window = Window.partitionBy("city").orderBy(
    col("call_drop_count").desc(),
    col("avg_latency").desc(),
    col("avg_download_speed").asc()
)

subscriber_ranked = subscriber_city_df.withColumn(
    "experience_rank",
    row_number().over(subscriber_window)
)
subscriber_ranked.show()

+-------------+--------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+-----------+------------------+-----------+---------------+---------------+
|subscriber_id|event_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|   event_time_clean|event_count|avg_download_speed|avg_latency|call_drop_count|experience_rank|
+-------------+--------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+-----------+------------------+-----------+---------------+---------------+
|        S8378| E101984|    T952|BANGALORE|          5G|            -60|              82.85|             NULL|        83|       NO|2026-01-01 01:39:12|     ANDROID|2026-01-01 01:39:12|         14|             91.76|     

In [26]:
# 3. Signal deterioration detection using lag
signal_window = Window.partitionBy("tower_id").orderBy("event_time_clean")

df_signal_lag = df.withColumn(
    "prev_signal_strength",
    lag("signal_strength").over(signal_window)
).withColumn(
    "signal_drop",
    col("prev_signal_strength") - col("signal_strength")
)

df_signal_lag.show()

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+--------------------+-----------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|   event_time_clean|prev_signal_strength|signal_drop|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+--------------------+-----------+
| E100028|        S9371|    T102|BANGALORE|          5G|            -62|              52.78|            45.88|       285|       NO|2026-01-01 00:01:24|         IOS|2026-01-01 00:01:24|                NULL|       NULL|
| E101314|        S5759|    T102|   MUMBAI|          3G|            -81|              38.74|            17.97|       278|       

## **PHASE 7: ANOMALY DETECTION**

In [27]:
# Detect towers where: Latency spikes, Download speed drops, Call drops spikes
rolling_window = Window.partitionBy("tower_id") \
    .orderBy("event_time_clean") \
    .rowsBetween(-3, -1)

df_anomaly = df.withColumn(
    "rolling_avg_latency",
    avg("latency_ms").over(rolling_window)
).withColumn(
    "rolling_avg_download",
    avg("download_speed_mbps").over(rolling_window)
).withColumn(
    "latency_spike",
    col("latency_ms") > col("rolling_avg_latency") * 1.5
).withColumn(
    "download_drop",
    col("download_speed_mbps") < col("rolling_avg_download") * 0.5
).withColumn(
    "call_drop_spike",
    col("call_drop") == "YES"
)

anomalous_events = df_anomaly.filter(
    col("latency_spike") | col("download_drop") | col("call_drop_spike")
)

anomalous_events.show()

+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+-------------------+--------------------+-------------+-------------+---------------+
|event_id|subscriber_id|tower_id|     city|network_type|signal_strength|download_speed_mbps|upload_speed_mbps|latency_ms|call_drop|         event_time| device_type|   event_time_clean|rolling_avg_latency|rolling_avg_download|latency_spike|download_drop|call_drop_spike|
+--------+-------------+--------+---------+------------+---------------+-------------------+-----------------+----------+---------+-------------------+------------+-------------------+-------------------+--------------------+-------------+-------------+---------------+
| E103037|        S7132|    T102|HYDERABAD|          5G|            -92|              27.06|             4.23|       257|       NO|2026-01-01 02:31:51|FEATUREPHONE|2026-01-01 02:31:51| 223.6